<a href="https://colab.research.google.com/github/say2imran/pyspark_salary_analysis/blob/main/pyspark_salary_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xvf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"


import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [10]:
mydata = spark.read.format("csv").option("header", "true").load("employe-salary-region-dummy.csv")


In [11]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [12]:
from pyspark.sql.functions import *

In [13]:
mydata2 = mydata.filter(mydata.JobTitle.isNotNull())

In [14]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Pr

In [15]:
mydata2 = mydata2.withColumn('clean_salary', mydata2.Salary.substr(2,100).cast('float'))

In [16]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [17]:
import pyspark.sql.functions as sqlfunc

In [18]:
# prompt: pyspark dataframe groupby aggregate by column and calculate average

average_salary = mydata2.groupBy("JobTitle").agg(sqlfunc.avg("clean_salary").alias("Average Salary"))
average_salary.show()


+--------------------+------------------+
|            JobTitle|    Average Salary|
+--------------------+------------------+
|Systems Administr...|  66131.4248046875|
|   Media Manager III|46968.356770833336|
|  Recruiting Manager|61231.947591145836|
|       Geologist III|    44579.80078125|
|        Geologist II|   43293.865234375|
|Database Administ...|     52018.4609375|
|   Financial Analyst|   62959.833203125|
|  Analyst Programmer|     37449.0921875|
|Software Engineer II|      74782.640625|
|       Accountant IV|   82732.248046875|
|    Product Engineer|    62290.42734375|
|Software Test Eng...|  59341.0712890625|
|Safety Technician...|   29421.529296875|
|    Junior Executive|    65262.55078125|
|Systems Administr...|       77059.21875|
|Human Resources A...| 40631.93994140625|
|        VP Marketing|60825.713216145836|
|  Environmental Tech|59367.870768229164|
|Mechanical System...|         75692.375|
| Assistant Professor|    49088.72421875|
+--------------------+------------

In [19]:
# prompt: pyspark search average salary based on gender column

average_salary_by_gender = mydata2.groupBy("Gender").agg(sqlfunc.avg("clean_salary").alias("Average Salary"))
average_salary_by_gender.show()


+------+------------------+
|Gender|    Average Salary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [20]:
# prompt: pyspark show min, max, average salary for each gender

min_salary_by_gender = mydata2.groupBy("Gender").agg(sqlfunc.min("clean_salary").alias("Min Salary"))
max_salary_by_gender = mydata2.groupBy("Gender").agg(sqlfunc.max("clean_salary").alias("Max Salary"))

average_salary_by_gender.show()
min_salary_by_gender.show()
max_salary_by_gender.show()


+------+------------------+
|Gender|    Average Salary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+

+------+----------+
|Gender|Min Salary|
+------+----------+
|Female|  10616.44|
|  Male|  10101.92|
+------+----------+

+------+----------+
|Gender|Max Salary|
+------+----------+
|Female|  99948.28|
|  Male|  99942.92|
+------+----------+



In [21]:
# prompt: pyspark average min max salary based on jobtitle

average_salary_by_jobtitle = mydata2.groupBy("JobTitle").agg(sqlfunc.avg("clean_salary").alias("Average Salary"),
                                                            sqlfunc.min("clean_salary").alias("Min Salary"),
                                                            sqlfunc.max("clean_salary").alias("Max Salary"))
average_salary_by_jobtitle.show()


+--------------------+------------------+----------+----------+
|            JobTitle|    Average Salary|Min Salary|Max Salary|
+--------------------+------------------+----------+----------+
|Systems Administr...|  66131.4248046875|  30266.46|  94859.08|
|   Media Manager III|46968.356770833336|  18453.02|  70306.29|
|  Recruiting Manager|61231.947591145836|  31690.72|  97121.81|
|       Geologist III|    44579.80078125|  17134.96|  95247.14|
|        Geologist II|   43293.865234375|  38597.39|  47990.34|
|Database Administ...|     52018.4609375|  52018.46|  52018.46|
|   Financial Analyst|   62959.833203125|  26980.55|  93817.51|
|  Analyst Programmer|     37449.0921875|  16987.33|  74349.58|
|Software Engineer II|      74782.640625|  74782.64|  74782.64|
|       Accountant IV|   82732.248046875|  65520.45|  99944.05|
|    Product Engineer|    62290.42734375|  24608.84|  97140.86|
|Software Test Eng...|  59341.0712890625|  24315.72|  85459.88|
|Safety Technician...|   29421.529296875